In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [2]:
chrome=''
def get_chrome():
    global chrome
    chrome=webdriver.Chrome(service=Service(r'C:\webdriver\chromedriver.exe'))
    chrome.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
    chrome.maximize_window()
    time.sleep(5)
    chrome.find_element(by=By.XPATH, value='/html/body/div[1]/main/div[4]/div/p[1]/a[1]').click()
    time.sleep(5)
    soup=BeautifulSoup(chrome.page_source,'lxml')
    return soup

'''
這個寫法是可以讓網頁停留直到你要的特定資料出現為止
element=WebDriverWait(chrome, 20).until(
EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[4]/div/p[1]/a[1]'))).click()
要引入的套件有二個
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
'''

"\n這個寫法是可以讓網頁停留直到你要的特定資料出現為止\nelement=WebDriverWait(chrome, 20).until(\nEC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[4]/div/p[1]/a[1]'))).click()\n要引入的套件有二個\nfrom selenium.webdriver.support.ui import WebDriverWait\nfrom selenium.webdriver.support import expected_conditions as EC\n"

In [3]:
soup=get_chrome()

In [4]:
datas=[]
for tr in (soup.find('tbody').find_all('tr')):
    data=[]
    for i,td in enumerate(tr.find_all('td')[:5]):
        if i==0:
            data.append(td.find('div',class_="hidden-phone print_show xrt-cur-indent").text.strip())
        else:  
            data.append(td.text.strip())
    if data!={}:
        datas.append([data[0],data[3],data[4]])
        # 把'-'取代成0
        datas = [[0 if val == '-' else val for val in i] for i in datas]
        # 把數字先轉成浮點數等一下要算均價
        datas = [[item if idx == 0 else float(item) if isinstance(item, str) and item.replace('.', '', 1).isdigit() else item for idx, item in enumerate(sublist)] for sublist in datas]
#         datas = [[*sublist, sum(sublist[1:]) / len(sublist[1:])] if isinstance(sublist[1], (float, int)) else sublist for sublist in datas]


datas
chrome.close()

In [5]:
df=pd.DataFrame(datas,columns=['幣別','即期買入','即期賣出'])
df = df.iloc[:,[1, 2]].apply(pd.to_numeric, errors='coerce')
average_values=df.mean(axis=1).round(5)
df['中價'] = average_values
df.insert(0,'幣別',['美金','港幣','英鎊','澳幣','加拿大幣','新加坡幣','瑞士法郎','日圓','南非幣','瑞典幣','紐元','泰幣',
                               '菲國比索','印尼幣','歐元','韓元','越南盾','馬來幣','人民幣'])
df

,幣別,即期買入,即期賣出,中價
0,美金,31.1600,31.3100,31.2350
1,港幣,3.9690,4.0390,4.0040
2,英鎊,39.3150,39.9450,39.6300
3,澳幣,20.7450,21.0900,20.9175
4,加拿大幣,23.1900,23.5200,23.3550
5,新加坡幣,23.3400,23.5600,23.4500
6,瑞士法郎,35.6900,36.0800,35.8850
7,日圓,0.2174,0.2224,0.2199
8,南非幣,1.6580,1.7480,1.7030
9,瑞典幣,2.9800,3.1000,3.0400
